<a href="https://colab.research.google.com/github/Havish77/AIML/blob/main/the_saga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import cv2 as cv
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

In [3]:

train_2 = '/content/drive/MyDrive/the saga'
ds_train_ = tf.keras.utils.image_dataset_from_directory(
    train_2,
    labels='inferred',
    label_mode='binary',
    image_size=[150, 150],
    interpolation='nearest',
    batch_size=32,
    shuffle=True
)

# Calculate the split sizes
total_batches = len(ds_train_)
train_split = int(0.96 * total_batches)  # 90% for training

# Create training and validation subsets
ds_train = ds_train_.take(train_split)  # First 90%
ds_valid = ds_train_.skip(train_split)  # Remaining 10%

# Optional: Prefetch for performance
ds_train = ds_train.prefetch(buffer_size=tf.data.AUTOTUNE)
ds_valid = ds_valid.prefetch(buffer_size=tf.data.AUTOTUNE)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
data_train = (
    ds_train
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

data_valid = (
    ds_valid
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

Found 590 files belonging to 2 classes.


In [5]:
model_2 = tf.keras.Sequential()
# Convolutional layers
model_2.add(tf.keras.layers.Conv2D(64, kernel_size=(5, 5), activation='relu', input_shape=(150,150, 3)))
model_2.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model_2.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'))
model_2.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_2.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu'))
model_2.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_2.add(tf.keras.layers.BatchNormalization())

model_2.add(tf.keras.layers.Flatten())
model_2.add(tf.keras.layers.Dense(512, activation='relu'))
model_2.add(tf.keras.layers.Dropout(0.5))
model_2.add(tf.keras.layers.Dense(256, activation='relu'))
model_2.add(tf.keras.layers.Dropout(0.3))
model_2.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model_2.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=['accuracy'])

model_2.fit(data_train,validation_data=data_valid,epochs=14,verbose=1)

Epoch 1/14
18/18 ━━━━━━━━━━━━━━━━━━━━ 65s 3s/step - accuracy: 0.6272 - loss: 0.9118 - val_accuracy: 0.5714 - val_loss: 0.7036
Epoch 2/14
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 106ms/step - accuracy: 0.6710 - loss: 0.7655 - val_accuracy: 0.7143 - val_loss: 0.6619
Epoch 3/14
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step - accuracy: 0.8414 - loss: 0.3979 - val_accuracy: 1.0000 - val_loss: 0.6462
Epoch 4/14
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - accuracy: 0.9103 - loss: 0.3041 - val_accuracy: 0.7857 - val_loss: 0.6291
Epoch 5/14
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step - accuracy: 0.9603 - loss: 0.1793 - val_accuracy: 1.0000 - val_loss: 0.6069
Epoch 6/14
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - accuracy: 0.9644 - loss: 0.1122 - val_accuracy: 0.8571 - val_loss: 0.5692
Epoch 7/14
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step - accuracy: 0.9760 - loss: 0.0837 - val_accuracy: 1.0000 - val_loss: 0.5195
Epoch 8/14
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - accuracy: 0.9738 - loss: 0.0850 - val_accuracy: 1.0

In [9]:
model_2.save('the_saga_final.keras')

In [11]:

from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.models import load_model
import os
from PIL import Image, UnidentifiedImageError
import numpy as np

# Custom Dataset Class
class TestImageDataset:
    def __init__(self, image_paths, transform=None):  # Corrected __init__ method
        self.image_paths = [path for path in image_paths if self.is_valid_image(path)]
        self.transform = transform

    def is_valid_image(self, img_path):
        try:
            Image.open(img_path).convert("RGB")
            return True
        except UnidentifiedImageError:
            print(f"Skipping invalid image: {img_path}")
            return False

    def __len__(self):  # Corrected __len__ method
        return len(self.image_paths)

    def __getitem__(self, idx):  # Corrected __getitem__ method
        img_path = self.image_paths[idx]
        image = keras_image.load_img(img_path, target_size=(224, 224))  # Adjust target size as needed
        image_array = keras_image.img_to_array(image) / 255.0  # Normalize to [0, 1]

        if self.transform:
            image_array = self.transform(image_array)

        return image_array, os.path.basename(img_path)



folder_path = '/content/drive/MyDrive/Test_Images'
image_paths = [os.path.join(folder_path, fname) for fname in os.listdir(folder_path)]

# Load the test dataset
test_dataset = TestImageDataset(image_paths)

# Convert to TensorFlow Dataset
def generator():
    for idx in range(len(test_dataset)):
        yield test_dataset[idx]

test_tf_dataset = tf.data.Dataset.from_generator(
    generator,
    output_signature=(
        tf.TensorSpec(shape=(150, 150, 3), dtype=tf.float32),  # Adjust shape as needed
        tf.TensorSpec(shape=(), dtype=tf.string)
    )
).batch(32)

# Load your trained model
#model_path = 'C:/Users/ponmu/OneDrive/Desktop/aiml_ps/my_model.keras'
#model = load_model(model_path)
# Evaluate the model and make predictions
predictions = []

for batch_images, filenames in test_tf_dataset:
    outputs = model_2(batch_images, training=False)
    for filename, output in zip(filenames.numpy(), outputs.numpy()):
        label = 1 if output >= 0.5 else 0  # Sigmoid output for binary classification
        label_str = 'Real' if label == 1 else 'AI'
        predictions.append((filename.decode("utf-8"), label_str))

# Print predictions
for prediction in predictions:
    print(prediction)


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element of shape (224, 224, 3) where an element of shape (150, 150, 3) was expected.
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (224, 224, 3) where an element of shape (150, 150, 3) was expected.


	 [[{{node PyFunc}}]] [Op:IteratorGetNext] name: 

In [ ]:
import csv

processed_predictions = [("image_" + str(int(filename.split('_')[1].split('.')[0])), label) for filename, label in predictions]
processed_predictions.sort()


# Save to CSV file
csv_path = 'img.csv'  # Save path can be adjusted
with open(csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'Label'])  # Write header
    writer.writerows(processed_predictions)  # Write data

# Print CSV location
print(f"Predictions saved to: {csv_path}")

# Print predictions for confirmation
for prediction in processed_predictions:
    print(prediction)